<a href="https://colab.research.google.com/github/kclaudeeager/ASP_Assignment2/blob/main/IMNEST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets

In [ ]:
# prompt: give me the starting for pytorch

# Load the data
train_set = datasets.MNIST(root='./data', train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

test_set = datasets.MNIST(root='./data', train=False, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

# Create data loaders
train_dataset = torch.utils.data.DataLoader(train_set, batch_size=64,
                                          shuffle=True, num_workers=4)

test_dataset = torch.utils.data.DataLoader(test_set, batch_size=64,
                                         shuffle=False, num_workers=4)


In [ ]:
import matplotlib.pyplot as plt
for data in train_dataset:
  print(data[0].shape)
  plt.imshow(data[0][8].view([28,28]))
  plt.show()
  break


In [ ]:
# prompt: #Analyse the balance of the dataset

from collections import Counter
print('Total number of training images:', len(train_set))
print('Total number of test images:', len(test_set))
print('Number of training images per class:', Counter(train_set.targets))
print('Number of test images per class:', Counter(test_set.targets))

total =0
counter_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}

for data in train_dataset:
  xs,ys = data

  for y in ys:
    counter_dict[int(y)] += 1
    total += 1
print(counter_dict)


In [ ]:
# prompt: Get the percentage of each image in dataset

for key,value in counter_dict.items():
  print(f"The percentage of {key} in the dataset is {round(value/total*100,2)}%")


In [ ]:
# prompt: show distinct image distribution for unique classes

plt.bar(counter_dict.keys(),counter_dict.values())
plt.xlabel("Class")
plt.ylabel("Number of Images")
plt.title("Distribution of Images per Class")
plt.show()


In [ ]:
# prompt: Generate the bar plot showing the distribution of the dataset, each image with it's count in form of percentages starting from here: counter_dict_percent = [{key:round(value/total*100,2)} for key,value in counter_dict.items()] print(counter_dict_percent)

import matplotlib.pyplot as plt
counter_dict_percent = {}
for key,value in counter_dict.items():
   counter_dict_percent.update({key:round(value/total*100,2)})
plt.bar(counter_dict_percent.keys(),counter_dict_percent.values())
plt.xlabel("Digits")
plt.ylabel("Percentage")
plt.title("Distribution of Digits in the MNIST Dataset")
plt.show()


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Build the NN
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    # define the input as the flattened shape of our data 28/28 and define the output (hidden layers to be 64)
    self.fc1 = nn.Linear(28*28,64)
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64,64)
    # last layer gets the output of 10 classes
    self.fc4 = nn.Linear(64,10)
   #(FFNN) feed forward NN
  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    # Distributing the classes (dim=1)
    return F.log_softmax(x,dim=1)
net = Net()
print(net)

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(),lr=0.001)
EPOCHS = 3
for epoch in range(EPOCHS):
  for data in train_dataset:
    X,y = data
    net.zero_grad()
    output = net(X.view([-1,28*28]))
    # Use nll_los because the output is a scaller not a vector else use mse
    loss = F.nll_loss(output,y)
    loss.backward() # Back propagation
    optimizer.step()# Adjust weights
  print(f"Loss as epoch {epoch}:{loss}")





In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for data in train_dataset:
    X,y= data
    net.zero_grad()
    output = net(X.view([-1,28*28]))
    correct += output.argmax(dim=1).eq(y).sum().item()
    total += y.size(0)


In [ ]:
print(total)
print(correct)
print(f"Accuracy is : {round(correct/total,3)}")

In [ ]:
plt.imshow(X[0].view(28,28))
plt.show()

In [ ]:
print(torch.argmax(net(X[0].view(-1,28*28))[0]))